# Python 包

TODO：还需要找一些资料来完善这部分，这部分目前逻辑还不完整

之前要使用别人的python包时，使用conda或者pip就能轻松的安装好，不需要从源码开始，很方便。然后写自己的python包时直接调用它们就行了，对于自己构建的python模块，也是直接调用就行。但是，如果你想要让自己的程序让程序之外整个系统上都能随意使用时，当你想让别人也能轻松地使用你的module时，这就不足够了。前者通常要求我们向module增加一个setup.py文件，后者则需要我们将程序打包发送到PyPI（这样别人就能pip安装了）。

下面看看相关具体内容。

主要参考了：

- [Writing a Python Package](https://towardsdatascience.com/writing-a-python-package-561146e53351)
- [What is setup.py?](https://stackoverflow.com/questions/1471994/what-is-setup-py)
- [the-hitchhikers-guide-to-packaging](https://the-hitchhikers-guide-to-packaging.readthedocs.io/en/latest/quickstart.html)
- [Building and Distributing Packages with Setuptools](https://setuptools.pypa.io/en/latest/setuptools.html)
- [Distributing Python Modules](https://docs.python.org/3/distutils/index.html)

## 简单的例子

setup.py文件是pip会在给定文件夹中寻找的文件。它使用能用来执行打包的setuptools工具。它其中内容包括包的名称，包的简短描述以及作者的信息，还有python版本。

本文件夹下有一个示例setup.py文件。

然后我们就可以尝试用用它，在本文件夹下打开terminal，执行：

```Shell
pip install .
```

此时输入：

```Shell
pip list
```

应该能看到有一个hello包。

在执行pip install . 语句的过程中，会创建一个wheel文件以用于发布包，然后使用wheel文件安装包到site-package文件夹中，如果要从网上下载其他包，则会在pkgs文件夹中创建缓存。

pip实际上就是一个从PyPI（Python Package Index）安装包的包安装器，是基于setuptools构建的。任何人都可以提交自己的包到PyPI上，全球的人都可以通过pip install your-package-name 来安装你上传的包。

## Packaging

从用户角度描述使用Distribution Utilities程序（“Distutils”）及其扩展在 Python 打包中的当前状态，描述如何通过构建包和安装第三方来扩展标准 Python 的功能：第三方包、模块和扩展。

Python 以其“batteries included”的理念而闻名，并拥有丰富的标准库。不过，作为一种流行语言，第三方包的数量远远大于标准库包的数量。因此，最终有必要了解如何在 Python 中使用、查找和创建包。

手动安装需要的额外软件包可能很乏味。因此，Python 有一个打包系统，允许人们以标准格式分发他们的程序和库，以便于安装和使用它们。除了分发包之外，Python 还为贡献包提供了一个中心服务。此服务称为 Python 包索引 (PyPI)。

一个package就是指一个拥有 \_\_init\_\_.py 文件的文件夹，比如本文件夹下执行：

```Shell
$ mkdir mypackage
$ cd mypackage
$ type nul > __init__.py
$ echo "# A Package" > __init__.py
$ cd ..
```

这会创建一个能被导入的包。

In [2]:
import mypackage
mypackage.__file__

'C:\\Projects\\git\\hydrus\\1-basic-envir\\mypackage\\__init__.py'

在大多数情况下，在当前工作目录中使用包仅适用于小型项目。在为较大的系统分发包时，使用工作目录作为包位置通常会成为一个问题。因此，创建 distutils 以将软件包轻松安装到 PYTHONPATH 中。 PYTHONPATH，也是代码中的 sys.path，是用于查找 Python 包的位置列表。例如：

In [4]:
import sys
sys.path

['C:\\Projects\\git\\hydrus\\1-basic-envir',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\python39.zip',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\DLLs',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\lib',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus',
 '',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\lib\\site-packages',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\lib\\site-packages\\win32',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\hust2\\miniconda3\\envs\\hydrus\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\hust2\\.ipython']

In [5]:
import mypackage
mypackage.__file__

'C:\\Projects\\git\\hydrus\\1-basic-envir\\mypackage\\__init__.py'

sys.path 中 null 或空字符串是当前工作目录，它允许找到当前工作目录中的包。

手动安装包的约定方式是将它们放在 .../site-packages/ 目录中。但是人们可能希望将 Python 模块安装到某个任意目录中。例如，您的site可能有一个约定，将所有与 Web 服务器应用程序相关的软件保存在 /www 下。附加 Python 模块可能属于 /www/pythonx.y/，为了导入它们，必须将此目录添加到 sys.path。有几种不同的方法可以添加目录。

最方便的方法是将路径配置文件添加到 Python 路径中已有的目录中，该目录可以是 .../site-packages/ 目录。路径配置文件的扩展名为 .pth，每行必须包含一个将附加到 sys.path 的路径。 （因为新路径被附加到 sys.path，添加目录中的模块不会覆盖标准模块。这意味着你不能使用这种机制来安装标准模块的固定版本。）

路径可以是绝对的或相对的，在这种情况下，它们相对于包含 .pth 文件的目录。

另外还有两个环境变量可以修改sys.path。 PYTHONHOME 为 Python 安装的前缀设置一个替代值。例如，如果 PYTHONHOME 设置为 /www/python/lib/python2.6/，则搜索路径将设置为 ['', '/www/python/lib/python2.6/', ...]。

PYTHONPATH 变量可以设置为将添加到 sys.path 开头的路径列表。例如，如果 PYTHONPATH 设置为 /www/python:/opt/py，则搜索路径将以 ['', '/www/python', '/opt/py', ...] 开头。

最后， sys.path 只是一个常规的 Python 列表，因此任何 Python 应用程序都可以通过添加或删除条目来修改它。

Python 安装在模块目录中有一个 site-packages 目录。此目录是放置用户安装的软件包的地方。维护此目录中的 .pth 文件，其中包含安装额外包的目录的路径。

虽然可以解压 tarball 并手动将它们放入 Python 安装目录（请参阅显式包含包位置），但使用包管理系统会给您带来一些显着的好处。以下是一些显而易见的：

- 依赖管理
    通常，您要安装的软件包需要其他包。包管理系统可以自动解决依赖关系，让您的安装轻松快捷。这是 distutils 提供的基本设施之一。但是，distutils 的其他扩展在安装依赖项方面做得更好。 （见Distribute）
- Accounting
    包管理器可以维护安装的东西列表和其他元数据，如安装的版本等。这使得用户很容易知道他的系统有什么东西。 （请参阅 Pip 安装 Python (Pip)）
- 卸载
    包管理器可以为您提供从环境中删除包的按钮方式。 （请参阅 Pip 安装 Python (Pip)）
- 搜索
    通过搜索特定术语的包索引来查找包。 （请参阅 Pip 安装 Python (Pip)）

distutils 模块是标准库的一部分，直到 Python 3.3。 distutils 模块已经在 Python 3.3 中停止使用。 distutils2（注意第二个）将向后兼容 Python 2.4 以后的版本；并成为 Python 3.3 标准库的一部分。

distutils 模块提供了打包 Python 的基础知识。不幸的是，distutils 模块充满了问题。

使用 virtualenv 创建微生态系统

在这里，我们有一个小题外话来简要讨论虚拟环境，这将在本指南的后面部分进行介绍。在大多数情况下，site-packages 目录是系统 Python 安装的一部分，非特权用户不可写入。此外，对我们可以使用的语言进行可靠的安装也很有用。如果我们想保持这种质量，实验包不应该与稳定的包混合在一起。为了实现这一点，大多数 Python 开发人员使用 virtualenv 包，它允许人们创建 Python 的虚拟安装。这将复制用户可写区域中的 site-packages 目录。位于虚拟环境中的 site-packages 目录是对全局目录的补充。虽然与整个软件包安装过程正交，但它是一种非常有用和自然的工作方式，因此将再次提及整个事情。 virtualenv 的安装和使用在 Virtual Environments 文档中有介绍。